In [1]:
#%reload_ext dotenv
#%dotenv

# ML libraries
import torch
import torch.nn as nn

# Local imports
from env import Env
from agents.random_agent import RandomAgent
from agents.minimax_agent import MinimaxAgent
from agents.minimax_agent_old import OldMinimaxAgent
#from agents.deep_q_agent import DeepQAgent
from agents.deep_q_agent_modified import DeepQAgent
from agents.cql_agent import CQLAgent
import utils
from trainer import Trainer

In [2]:
##################
# INITIALIZATION #
##################

# Fix random seed
utils.seed_everything(42, deterministic=False)

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define player and opponent IDs
AGENT = 1
OPPONENT = 2

# Define game environment, this should be passed to the agents and the trainer
env = Env()

# Define agent and opponent
agent = DeepQAgent(env=env, epsilon_max=1, epsilon_min=0.1, epsilon_decay=0.9999, device=device)
opponent = MinimaxAgent(env=env, depth=3, epsilon=0.5, player=OPPONENT)
replacement_agent = MinimaxAgent(env=env, depth=3, epsilon=0.5, player=OPPONENT)

# Define options for training
options = {
           'UPDATE_OPPONENT': True,                         # Whether to enable self-play or not
           'OPPONENT_UPDATE_FREQUENCY': 100,                # After how many episodes the opponent will be replaced by the current agent
           'BOOTSTRAP_EPISODES': 7500,                      # During this time, the agent will not be replaced by itself
           'DECAY_RANDOMNESS_OPPONENT': True,               # Decay randomness of the opponent. Use only if the opponent acts with some randomness
           'DECAY_RANDOMNESS_FREQUENCY': 1000,              # Frequency of randomness decay
           'REPLACE_FOR_EVALUATION': True,                  # Whether to replace the training model at the end with another evaluation model
           'REPLACE_FOR_EVALUATION_BY': replacement_agent,  # Evalutation model to replace training model by
           'AUTOSAVE': True,                                # Whether to save the model at certain intervals
           'AUTOSAVE_TYPE': 'NUM_EPISODES',                 # One of ["NUM_OPTIMIZATIONS", "NUM_EPISODES"]
           'AUTOSAVE_PERIOD': 1000,                         # After how many _ to save the model
           }

# Initialize trainer
trainer = Trainer(env=env, agent=agent, opponent=opponent, agent_id=AGENT, opponent_id=OPPONENT, num_episodes={'TRAIN': 10000, 'EVAL': 100}, device=device, verbose=True)

/home/albin/miniconda3/envs/dl-proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
############
# TRAINING #
############

# Train agent
trainer.train()

In [3]:
####################
# QUICK EVALUATION #
####################

# Fix random seed
utils.seed_everything(42, deterministic=False)

# Define player and opponent IDs
AGENT = 1
OPPONENT = 2

# Fix random seed
utils.seed_everything(42, deterministic=False)

eval_env = Env()
# Load agent from save and set to eval mode
minmax_agent = MinimaxAgent(env=eval_env, depth=5, epsilon=0.0, player=AGENT)

# Define opponent
minmax_opponent = MinimaxAgent(env=eval_env, depth=5, epsilon=0.0, player=OPPONENT)

# Initialize trainer
trainer = Trainer(env=eval_env, agent=agent, opponent=opponent, agent_id=AGENT, opponent_id=OPPONENT, num_episodes={'TRAIN': 10000, 'EVAL': 100}, device=device, verbose=True)
# Run evaluation
trainer.eval(agent=minmax_agent, opponent=minmax_opponent, episodes=2, agent_start=None, print_last_n_games=2)

EVAL: Running episode 1 of 2. Ratios are [WINS: 0.00% | LOSSES: 0.00% | TIES: 100.00%]This is inside Trainer: <env.Env object at 0x7f165a6dfcd0>
This is inside minimax_agent: <env.Env object at 0x7f165a6dfcd0>


_______________
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|1|0|0|0|
AGENT action was 3
This is inside minimax_agent: <env.Env object at 0x7f165a6dfcd0>


_______________
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|2|0|0|0|
|0|0|0|1|0|0|0|
OPPONENT action was 3
This is inside Trainer: <env.Env object at 0x7f165a6dfcd0>
This is inside minimax_agent: <env.Env object at 0x7f165a6dfcd0>


_______________
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|2|0|0|0|
|0|0|0|1|1|0|0|
AGENT action was 4
This is inside minimax_agent: <env.Env object at 0x7f165a6dfcd0>


_______________
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|
|0|0|0|2|0|0|0|
|0|0|2|1|1|0|0|
OPPONENT action was 

KeyboardInterrupt: 